In [1]:
using HypothesisTests, Distributions
n = 50
x = 0.5 .^ (1:n)

50-element Vector{Float64}:
 0.5
 0.25
 0.125
 0.0625
 0.03125
 0.015625
 0.0078125
 0.00390625
 0.001953125
 0.0009765625
 0.00048828125
 0.000244140625
 0.0001220703125
 ⋮
 1.8189894035458565e-12
 9.094947017729282e-13
 4.547473508864641e-13
 2.2737367544323206e-13
 1.1368683772161603e-13
 5.684341886080802e-14
 2.842170943040401e-14
 1.4210854715202004e-14
 7.105427357601002e-15
 3.552713678800501e-15
 1.7763568394002505e-15
 8.881784197001252e-16

In [2]:
d = DiscreteNonParametric(1:n, x ./ sum(x))

DiscreteNonParametric{Int64, Float64, UnitRange{Int64}, Vector{Float64}}(
support: 1:50
p: [0.5000000000000004, 0.2500000000000002, 0.1250000000000001, 0.06250000000000006, 0.03125000000000003, 0.015625000000000014, 0.007812500000000007, 0.0039062500000000035, 0.0019531250000000017, 0.0009765625000000009  …  4.547473508864645e-13, 2.2737367544323226e-13, 1.1368683772161613e-13, 5.684341886080807e-14, 2.842170943040403e-14, 1.4210854715202016e-14, 7.105427357601008e-15, 3.552713678800504e-15, 1.776356839400252e-15, 8.88178419700126e-16]
)


In [18]:
display(methodswith(typeof(d)))
display(fieldnames(typeof(d)))

0-element Vector{Method}

(:support, :p)

In [23]:
fieldnames(DiscreteNonParametric)

(:support, :p)

In [22]:
using Plots;
histogram(x)

In [6]:
?DiscreteNonParametric

search: DiscreteNonParametric



```
DiscreteNonParametric(xs, ps)
```

A *Discrete nonparametric distribution* explicitly defines an arbitrary probability mass function in terms of a list of real support values and their corresponding probabilities

```julia
d = DiscreteNonParametric(xs, ps)

params(d)  # Get the parameters, i.e. (xs, ps)
support(d) # Get a sorted AbstractVector describing the support (xs) of the distribution
probs(d)   # Get a Vector of the probabilities (ps) associated with the support
```

External links

  * [Probability mass function on Wikipedia](http://en.wikipedia.org/wiki/Probability_mass_function)


In [3]:
sample = vcat(ones(Int64,16), 2*ones(Int64,8), 3*ones(Int64,4), 4,4,5)

31-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 2
 2
 2
 2
 2
 3
 3
 3
 3
 4
 4
 5

In [5]:
length(sample)

31

In [7]:
display(ExactOneSampleKSTest(sample, d))

┌ Warning: This test is inaccurate with ties
└ @ HypothesisTests /home/yh/.julia/packages/HypothesisTests/BgrVj/src/kolmogorov_smirnov.jl:68


Exact one sample Kolmogorov-Smirnov test
----------------------------------------
Population details:
    parameter of interest:   Supremum of CDF differences
    value under h_0:         0.0
    point estimate:          0.5

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-06

Details:
    number of observations:   31


In [40]:
display(pvalue(ExactOneSampleKSTest(sample, d), tail=:both))
display(pvalue(ExactOneSampleKSTest(sample, d), tail=:left))

display(pvalue(ExactOneSampleKSTest(sample, d), tail=:right))


┌ Warning: This test is inaccurate with ties
└ @ HypothesisTests /home/yh/.julia/packages/HypothesisTests/BgrVj/src/kolmogorov_smirnov.jl:68


1.0844646944913948e-7

┌ Warning: This test is inaccurate with ties
└ @ HypothesisTests /home/yh/.julia/packages/HypothesisTests/BgrVj/src/kolmogorov_smirnov.jl:68


5.422323472456974e-8

┌ Warning: This test is inaccurate with ties
└ @ HypothesisTests /home/yh/.julia/packages/HypothesisTests/BgrVj/src/kolmogorov_smirnov.jl:68


0.9213372964737401

In [24]:
?ExactOneSampleKSTest

search: ExactOneSampleKSTest



```
ExactOneSampleKSTest(x::AbstractVector{<:Real}, d::UnivariateDistribution)
```

Perform a one-sample exact Kolmogorov–Smirnov test of the null hypothesis that the data in vector `x` comes from the distribution `d` against the alternative hypothesis that the sample is not drawn from `d`.

Implements: [`pvalue`](@ref)


In [25]:
?ChisqTest

search: ChisqTest



```
ChisqTest(x[, y][, theta0 = ones(length(x))/length(x)])
```

Perform a Pearson chi-squared test (equivalent to a [`PowerDivergenceTest`](@ref) with $λ = 1$).

If `y` is not given and `x` is a matrix with one row or column, or `x` is a vector, then a goodness-of-fit test is performed (`x` is treated as a one-dimensional contingency table). In this case, the hypothesis tested is whether the population probabilities equal those in `theta0`, or are all equal if `theta0` is not given.

If `x` is a matrix with at least two rows and columns, it is taken as a two-dimensional contingency table. Otherwise, `x` and `y` must be vectors of the same length. The contingency table is calculated using `counts` function from the `StatsBase` package. Then the power divergence test is conducted under the null hypothesis that the joint distribution of the cell counts in a 2-dimensional contingency table is the product of the row and column marginals.

Note that the entries of `x` (and `y` if provided) must be non-negative integers.

Implements: [`pvalue`](@ref), [`confint`](@ref)


In [45]:
using StatsBase;
#countmap is from StatsBase
onesampleGOF(sample, d::DiscreteNonParametric) = begin
    cm = [get(StatsBase.countmap(sample),k,0) for k in d.support]
    sum(values(cm)) < length(sample) && error("Sample has elements outside support")
    return ChisqTest(cm, d.p)
end

onesampleGOF (generic function with 1 method)

In [46]:
pvalue(onesampleGOF(sample, d))

1.0

In [48]:
display(pvalue(onesampleGOF(fill(31,10), d)))
display(pvalue(onesampleGOF(rand(d,100), d)))
display(pvalue(onesampleGOF(rand(1:51,10), d)))

0.0

0.9999999999999981

0.0

In [51]:
count_dict = StatsBase.countmap(sample)

Dict{Int64, Int64} with 5 entries:
  5 => 1
  4 => 2
  2 => 8
  3 => 4
  1 => 16

In [53]:
count_dict[1]

16

In [57]:
cm = [get(StatsBase.countmap(sample),k,0) for k in d.support]

50-element Vector{Int64}:
 16
  8
  4
  2
  1
  0
  0
  0
  0
  0
  0
  0
  0
  ⋮
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0

In [62]:
display(length(sample))
display(sum(cm))

31

31

In [63]:
length(cm)

50

In [64]:
?ChisqTest

search: ChisqTest



```
ChisqTest(x[, y][, theta0 = ones(length(x))/length(x)])
```

Perform a Pearson chi-squared test (equivalent to a [`PowerDivergenceTest`](@ref) with $λ = 1$).

If `y` is not given and `x` is a matrix with one row or column, or `x` is a vector, then a goodness-of-fit test is performed (`x` is treated as a one-dimensional contingency table). In this case, the hypothesis tested is whether the population probabilities equal those in `theta0`, or are all equal if `theta0` is not given.

If `x` is a matrix with at least two rows and columns, it is taken as a two-dimensional contingency table. Otherwise, `x` and `y` must be vectors of the same length. The contingency table is calculated using `counts` function from the `StatsBase` package. Then the power divergence test is conducted under the null hypothesis that the joint distribution of the cell counts in a 2-dimensional contingency table is the product of the row and column marginals.

Note that the entries of `x` (and `y` if provided) must be non-negative integers.

Implements: [`pvalue`](@ref), [`confint`](@ref)


In [68]:
StatsBase.counts(cm)

17-element Vector{Int64}:
 45
  1
  1
  0
  1
  0
  0
  0
  1
  0
  0
  0
  0
  0
  0
  0
  1

In [69]:
StatsBase.counts(d.p)

LoadError: MethodError: no method matching counts(::Vector{Float64})
[0mClosest candidates are:
[0m  counts([91m::AbstractArray{T} where T<:Integer[39m) at ~/.julia/packages/StatsBase/XgjIN/src/counts.jl:91
[0m  counts([91m::AbstractArray{T} where T<:Integer[39m, [91m::AbstractWeights[39m) at ~/.julia/packages/StatsBase/XgjIN/src/counts.jl:92
[0m  counts([91m::AbstractArray{T} where T<:Integer[39m, [91m::UnitRange{T} where T<:Integer[39m) at ~/.julia/packages/StatsBase/XgjIN/src/counts.jl:85
[0m  ...